<a href="https://colab.research.google.com/github/yiluwang-am/yiluwang-am.github.io/blob/main/prodomino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi
import torch, sys
print("Torch:", torch.__version__, "| CUDA available:", torch.cuda.is_available())

Fri Nov  7 12:46:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
%cd /content
!rm -rf ProDomino
!git clone https://github.com/Niopek-Lab/ProDomino.git
%cd ProDomino

# Python deps (keep Colab's PyTorch)
!pip install -q fair-esm biopython einops tqdm matplotlib pandas

/content
Cloning into 'ProDomino'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 84 (delta 18), reused 70 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (84/84), 34.77 MiB | 24.69 MiB/s, done.
Resolving deltas: 100% (18/18), done.
/content/ProDomino


In [3]:
import sys, os
sys.path.append("/content/ProDomino/src")   # <- key line
print("In path?", "/content/ProDomino/src" in sys.path)
!ls -la /content/ProDomino/src

In path? True
total 12
drwxr-xr-x 3 root root 4096 Nov  7 12:47 .
drwxr-xr-x 7 root root 4096 Nov  7 12:47 ..
drwxr-xr-x 5 root root 4096 Nov  7 12:47 ProDomino


In [4]:
# 0) Pin numpy to 1.26.4 to avoid ABI mismatches
!pip install -q --upgrade --force-reinstall "numpy==1.26.4"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [5]:
# 1) Install the rest (all compatible with numpy 1.26)
!pip install -q \
  pandas==2.2.2 \
  matplotlib==3.8.1 \
  seaborn==0.13.1 \
  biopython==1.81 \
  torchmetrics==1.4.0 \
  lightning==2.3.0 \
  fair-esm==2.0.0 \
  fairscale==0.4.13 \
  fsspec==2024.3.0 \
  einops tqdm

In [ ]:
import torch, glob
from ProDomino import Embedder, ProDomino  # now works because we added /src to sys.path

seq = "MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQANLKKKQLLEHVDKHF"  # replace with yours

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_grad_enabled(False)

embedder = Embedder()                     # auto-uses GPU if available
embedding = embedder.predict_embedding(seq)

ckpt_path = glob.glob("/content/ProDomino/checkpoints/*.pt")[0]
model = ProDomino(ckpt_path, "mini_3b_mlp")
scores = model.predict_insertion_sites(embedding)

print("len:", len(scores), "| first 10:", [round(float(x),4) for x in scores[:10]])

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t36_3B_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t36_3B_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t36_3B_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t36_3B_UR50D-contact-regression.pt
